In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

import main

In [41]:
## Start of Notebook

print('Starting Assignment 1')
RawData = main.ReadInData("../data/tcd ml 2019-20 income prediction training (with labels).csv")
print(RawData.shape)
print(RawData.describe())

## Might not need this line here. Just a precaution for NaN.
# if trainingData.isnull().any:
#     trainingData = trainingData.fillna(method='ffill')


Starting Assignment 1
(111993, 12)
            Instance  Year of Record            Age  Size of City  \
count  111993.000000   111552.000000  111499.000000  1.119930e+05   
mean    55997.000000     1999.421274      37.345304  8.388538e+05   
std     32329.738686       11.576382      16.036694  2.196879e+06   
min         1.000000     1980.000000      14.000000  7.700000e+01   
25%     27999.000000     1989.000000      24.000000  7.273400e+04   
50%     55997.000000     1999.000000      35.000000  5.060920e+05   
75%     83995.000000     2009.000000      48.000000  1.184501e+06   
max    111993.000000     2019.000000     115.000000  4.999251e+07   

       Wears Glasses  Body Height [cm]  Income in EUR  
count  111993.000000     111993.000000   1.119930e+05  
mean        0.500531        175.220192   1.092138e+05  
std         0.500002         19.913889   1.498024e+05  
min         0.000000         94.000000  -5.696906e+03  
25%         0.000000        160.000000   3.077169e+04  
50%    

In [42]:
## Dropping Features that are missing a large amount of data
## Dropping instances with missing data

RawData = main.HandleMissingData(RawData)

null_counts = RawData.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

37
1999.4212743832472
Number of null values in each column:
Year of Record      0
Age                 0
Country             0
Size of City        0
Profession          0
Wears Glasses       0
Body Height [cm]    0
Income in EUR       0
dtype: int64


In [43]:
## Checking data for missing values

null_counts = RawData.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))


Number of null values in each column:
Year of Record      0
Age                 0
Country             0
Size of City        0
Profession          0
Wears Glasses       0
Body Height [cm]    0
Income in EUR       0
dtype: int64


In [44]:
## Seeing the data and the unique values

for s in RawData.columns.values:
    print("Features in " + s)
    print(RawData[s].unique())
    print(len(RawData[s].unique()))


Features in Year of Record
[1997. 1996. 2018. 2006. 2010. 1991. 1982. 2008. 2015. 2019. 1981. 1989.
 2003. 1987. 1995. 1992. 1980. 1983. 2011. 2014. 1985. 2005. 1998. 2001.
 2017. 2007. 1988. 1994. 2012. 2009. 2013. 2004. 1999. 2002. 1993. 2000.
 1990. 2016. 1986. 1984.]
40
Features in Age
[ 41.  28.  33.  46.  60.  71.  36.  43.  16.  51.  15.  22.  32.  40.
  31.  27.  25.  35.  63.  38.  34.  20.  23.  14.  37.  62.  52.  47.
  24.  26.  50.  49.  18.  19.  39.  66.  21.  44.  29.  64.  65.  53.
  78.  42.  58.  17.  30.  79.  48.  56.  83.  54.  74.  87.  90.  67.
  45.  77.  73.  55.  92.  80.  72.  57.  70.  59.  61.  82.  69.  98.
  94.  68.  81.  97.  76.  75.  88.  91.  96.  84.  89. 107.  85.  86.
 100. 110.  95.  99.  93. 104. 102. 101. 105. 111. 108. 103. 109. 106.
 115.]
99
Features in Country
['Belarus' 'Singapore' 'Norway' 'Cuba' 'United Arab Emirates' 'Liberia'
 'State of Palestine' 'Israel' 'South Sudan' 'Kyrgyzstan' 'Togo' 'Finland'
 'Sierra Leone' 'Papua New Guinea' 

In [45]:
## Section to conver to binary matrix
## Changing Country, Profession

RawData_Country = pd.get_dummies(RawData.Country)
print(type(RawData_Country))
RawData_Profession = pd.get_dummies(RawData.Profession)
print(type(RawData_Profession))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [46]:
RawData.describe()

,Year of Record,Age,Size of City,Wears Glasses,Body Height [cm],Income in EUR
count,111993.000000,111993.000000,1.119930e+05,111993.000000,111993.000000,1.119930e+05
mean,1999.419616,37.343780,8.388538e+05,0.500531,175.220192,1.092138e+05
std,11.553597,16.001302,2.196879e+06,0.500002,19.913889,1.498024e+05
min,1980.000000,14.000000,7.700000e+01,0.000000,94.000000,-5.696906e+03
25%,1989.000000,24.000000,7.273400e+04,0.000000,160.000000,3.077169e+04
50%,1999.000000,35.000000,5.060920e+05,1.000000,174.000000,5.733917e+04
75%,2009.000000,48.000000,1.184501e+06,1.000000,190.000000,1.260936e+05
max,2019.000000,115.000000,4.999251e+07,1.000000,265.000000,5.285252e+06


In [47]:
## Creating clean training data
print(RawData.columns)

RawData = pd.concat([RawData, RawData_Country], axis=1)
RawData= RawData.drop('Country', axis=1)
RawData = pd.concat([RawData, RawData_Profession], axis=1)
RawData= RawData.drop('Profession', axis=1)

Index(['Year of Record', 'Age', 'Country', 'Size of City', 'Profession',
       'Wears Glasses', 'Body Height [cm]', 'Income in EUR'],
      dtype='object')


In [48]:
RawData.describe()

,Year of Record,Age,Size of City,Wears Glasses,Body Height [cm],Income in EUR,Afghanistan,Albania,Algeria,Angola,...,windows administrator,wireless coordinator,woodworker,word processor,workforce management analyst,workforce planning intern,writer,x-ray technician,yardmaster,youth initiatives lead advisor
count,111993.000000,111993.000000,1.119930e+05,111993.000000,111993.000000,1.119930e+05,111993.000000,111993.000000,111993.000000,111993.000000,...,111993.000000,111993.000000,111993.000000,111993.00000,111993.000000,111993.000000,111993.000000,111993.000000,111993.000000,111993.000000
mean,1999.419616,37.343780,8.388538e+05,0.500531,175.220192,1.092138e+05,0.000196,0.003554,0.000063,0.000473,...,0.000241,0.000241,0.000313,0.00025,0.000277,0.000214,0.000286,0.000268,0.000188,0.000286
std,11.553597,16.001302,2.196879e+06,0.500002,19.913889,1.498024e+05,0.014014,0.059508,0.007906,0.021749,...,0.015525,0.015525,0.017676,0.01581,0.016635,0.014637,0.016901,0.016365,0.013692,0.016901
min,1980.000000,14.000000,7.700000e+01,0.000000,94.000000,-5.696906e+03,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1989.000000,24.000000,7.273400e+04,0.000000,160.000000,3.077169e+04,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1999.000000,35.000000,5.060920e+05,1.000000,174.000000,5.733917e+04,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2009.000000,48.000000,1.184501e+06,1.000000,190.000000,1.260936e+05,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2019.000000,115.000000,4.999251e+07,1.000000,265.000000,5.285252e+06,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
## Output Training Data to new file
RawData.to_csv("../data/processedData.csv", index=False)

In [49]:
## Read in new data and model

CleanData = main.ReadInData("../data/processedData2.csv")

In [50]:
CleanData.head(5)

,Instance,Year of Record,Age,Size of City,Wears Glasses,Body Height [cm],Income in EUR,Afghanistan,Albania,Algeria,...,windows administrator,wireless coordinator,woodworker,word processor,workforce management analyst,workforce planning intern,writer,x-ray technician,yardmaster,youth initiatives lead advisor
0,1,1997.0,41.0,1239930,0,193,61031.94416,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1996.0,41.0,1603504,0,186,91001.32764,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,2018.0,28.0,1298017,1,170,157982.17670,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,2006.0,33.0,751903,1,171,45993.75793,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,2010.0,46.0,95389,0,188,38022.16217,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
income = CleanData['Income in EUR']
features = CleanData.drop(labels=['Income in EUR', 'Instance'], axis=1)

features.head(5)

,Year of Record,Age,Size of City,Wears Glasses,Body Height [cm],Afghanistan,Albania,Algeria,Angola,Argentina,...,windows administrator,wireless coordinator,woodworker,word processor,workforce management analyst,workforce planning intern,writer,x-ray technician,yardmaster,youth initiatives lead advisor
0,1997.0,41.0,1239930,0,193,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1996.0,41.0,1603504,0,186,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2018.0,28.0,1298017,1,170,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2006.0,33.0,751903,1,171,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2010.0,46.0,95389,0,188,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
## Splitting Data

features_train, features_val, income_train, income_val = train_test_split(features, income, test_size=0.2, random_state=0)


In [ ]:
## Using Linear Regression

model = LinearRegression()
model.fit(features_train, income_train)

print(model.coef_)
print(model.intercept_)


In [ ]:
## Predict with the validation data

income_predict = model.predict(features_val)
comparison = pd.DataFrame({'Actual': income_val, 'Predicted': income_predict})

comparison

In [ ]:
## Graphing Difference 

df1 = comparison.head(25)
df1.plot(kind='bar',figsize=(16,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(income_val, income_predict))  
print('Mean Squared Error:', metrics.mean_squared_error(income_val, income_predict))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(income_val, income_predict)))

## Mean Absolute Error: 43557.00458896904
# Mean Squared Error: 7422065078.367375
# Root Mean Squared Error: 86151.40787223025

In [ ]:
## Read in Test and try to map

TestData = main.ReadInData("../data/tcd ml 2019-20 income prediction test (without labels).csv")
TestData.head(5)


In [ ]:
## Comparing Training and Test (Country)
print(TestData['Country'].unique())
print("Test Shape " + str(len(TestData['Country'].unique())))
Raw = main.ReadInData("../data/tcd ml 2019-20 income prediction training (with labels).csv")
print(Raw['Country'].unique())
print("Raw Shape " + str(len(Raw['Country'].unique())))

print(TestData['Country'].equals(Raw['Country']))

## Test has less than 3 to Training

In [ ]:
## Comparing Training and Test (Profession)
print(TestData['Profession'].unique())
print("Test Shape " + str(len(TestData['Profession'].unique())))
print(Raw['Profession'].unique())
print("Raw Shape " + str(len(Raw['Profession'].unique())))

print(TestData['Profession'].equals(Raw['Profession']))

## Test Data has less 15 to Training

In [ ]:
## Comparing Training and Test (University Degree)
print(TestData['University Degree'].unique())
print("Test Shape " + str(len(TestData['University Degree'].unique())))
print(Raw['University Degree'].unique())
print("Raw Shape " + str(len(Raw['University Degree'].unique())))

print(TestData['University Degree'].equals(Raw['University Degree']))

## Test Data has less 15 to Training

In [ ]:
## Cell to Compare what is in Test and Trainning Data

Diff = pd.DataFrame(columns=['Unique_C_InTest', 'Unique_C_InTrain','Unique_P_InTest', 'Unique_P_InTrain'])

Diff = pd.DataFrame()
Diff['Unique_C_InTest'] = TestData["Country"][~TestData["Country"].isin(Raw["Country"])].drop_duplicates()
print(Diff['Unique_C_InTest'])
print(str(len(Diff['Unique_C_InTest'])))

Diff = pd.DataFrame()
Diff['Unique_P_InTest',] = TestData["Profession"][~TestData["Profession"].isin(Raw["Profession"])].drop_duplicates()
print(Diff['Unique_P_InTest',])
print(str(len(Diff['Unique_P_InTest',])))

Diff = pd.DataFrame()
Diff['Unique_C_InTrain'] = Raw["Country"][~Raw["Country"].isin(TestData["Country"])].drop_duplicates()
print(Diff['Unique_C_InTrain'])
print(str(len(Diff['Unique_C_InTrain'])))

Diff = pd.DataFrame()
Diff['Unique_P_InTrain'] = Raw["Profession"][~Raw["Profession"].isin(TestData["Profession"])].drop_duplicates()
print(Diff['Unique_P_InTrain'])
print(str(len(Diff['Unique_P_InTrain'])))


In [ ]:
Raw.columns.values

In [ ]:
Raw = main.HandleMissingData(Raw)

In [ ]:
null_counts = Raw.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

In [ ]:
TestData = main.HandleMissingData(TestData)
null_counts = TestData.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

In [52]:
## Cell used to test PrepTestForModel

train = main.ReadInData("../data/processedData2.csv")
Raw = main.ReadInData("../data/tcd ml 2019-20 income prediction training (with labels).csv")
test = main.ReadInData("../data/tcd ml 2019-20 income prediction test (without labels).csv")
    

In [74]:
## Cell used to test PrepTestForModel
test = main.ReadInData("../data/tcd ml 2019-20 income prediction test (without labels).csv")
test = main.HandleMissingData(test)
CleanTest = test.drop('Country', axis=1)
CleanTest = CleanTest.drop('Profession', axis=1)
CleanTest = CleanTest.drop('Income', axis=1)

# Create 1st DF as copy. Create 2nd DF with encoding. Cat together

print(CleanTest.columns.unique)
print(train.columns.unique)
countries = Raw['Country'].unique()
professions = Raw['Profession'].unique()

countries.sort()
# professions.sort()

array = [0] * len(countries)
zero_array_C = np.zeros((len(test['Country']), len(countries)))
zero_array_P = np.zeros((len(test['Profession']), len(professions)))

countryDf = pd.DataFrame(data=zero_array_C, columns=countries)
professionDf = pd.DataFrame(data=zero_array_P, columns=professions)

count = 0

series = pd.Series(array)

print(countryDf.describe())
print(professionDf.describe())
# for i in test['Country']:
#     countryDf.loc[count] = series
#     if i in countries:
#         # Write output
#         # "I think this is right... it is right"
#         countryDf.loc[count][i] = 1
        
#     count = count + 1
    

# print(countryDf)

37
1999.5144306574348
<bound method Index.unique of Index(['Year of Record', 'Age', 'Size of City', 'Wears Glasses',
       'Body Height [cm]'],
      dtype='object')>
<bound method Index.unique of Index(['Instance', 'Year of Record', 'Age', 'Size of City', 'Wears Glasses',
       'Body Height [cm]', 'Income in EUR', 'Afghanistan', 'Albania',
       'Algeria',
       ...
       'windows administrator', 'wireless coordinator', 'woodworker',
       'word processor ', 'workforce management analyst',
       'workforce planning intern', 'writer', 'x-ray technician', 'yardmaster',
       'youth initiatives lead advisor'],
      dtype='object', length=1508)>
       Afghanistan  Albania  Algeria   Angola  Argentina  Armenia  Australia  \
count      73230.0  73230.0  73230.0  73230.0    73230.0  73230.0    73230.0   
mean           0.0      0.0      0.0      0.0        0.0      0.0        0.0   
std            0.0      0.0      0.0      0.0        0.0      0.0        0.0   
min            0.0  

In [26]:
results = np.where(countries == 'Portugal')
print(results)
print(countries[22])

(array([22]),)
Portugal


In [59]:
print(countryDf)
print(count)
test = [0]*len(countries)
test_series = pd.Series(test)
test_series.describe()

      Belarus Singapore Norway Cuba United Arab Emirates Liberia  \
0         NaN       NaN    NaN  NaN                  NaN     NaN   
1         NaN       NaN    NaN  NaN                  NaN     NaN   
2         NaN       NaN    NaN  NaN                  NaN     NaN   
3         NaN       NaN    NaN  NaN                  NaN     NaN   
4         NaN       NaN    NaN  NaN                  NaN     NaN   
...       ...       ...    ...  ...                  ...     ...   
26682     NaN       NaN    NaN  NaN                  NaN     NaN   
26683     NaN       NaN    NaN  NaN                  NaN     NaN   
26684       1       NaN    NaN  NaN                  NaN     NaN   
26685     NaN       NaN    NaN  NaN                  NaN     NaN   
26686     NaN       NaN    NaN  NaN                  NaN     NaN   

      State of Palestine Israel South Sudan Kyrgyzstan  ... Micronesia  \
0                    NaN    NaN         NaN        NaN  ...        NaN   
1                    NaN    NaN    

count    160.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64